# Model Evaluation Script

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras import regularizers, optimizers
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from keras import backend as K
from sklearn.metrics import log_loss


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../Final"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


Data_Organization.ipynb
Final_Data_Organization.ipynb
FinalMarchMadnessAdvStats.csv
Final MLP.ipynb
MarchMadnessAdvStats.csv
MarchMadnessFeatures.csv
MarchMadnessTest.csv
OldWNCAATourneyCompactResults.csv
OldWNCAATourneySeeds.csv
wModel.py
WNCAATourneyCompactResults.csv
WNCAATourneyDetailedResults.csv
WNCAATourneySeeds.csv
WNCAATourneySlots.csv
WRegularSeasonCompactResults.csv
WRegularSeasonDetailedResults.csv
WSampleSubmissionStage2.csv
WSeasons.csv
WTeams.csv
WTeamSpellings.csv



## Load pre-organized data

In [2]:
data_dir = '../Final/'
df_features = pd.read_csv(data_dir + 'MarchMadnessFeatures.csv')

Format and scale the data

In [136]:
X = df_features.iloc[:,1:]
xDim = np.shape(X)[1]
X_train = X.values.reshape(-1,xDim)
y_train = df_features.Result.values

print('Feature vector dimension is: %.2f' % xDim)
# print(X_train)

Feature vector dimension is: 28.00


In [137]:
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
X_train = pca.fit_transform(X_train)

In [138]:
xDim = np.shape(X_train)[1]
print(np.shape(X_train))

(504, 10)


# Load Training Data


In [139]:
df_test = pd.read_csv(data_dir+'MarchMadnessTest.csv')

X_test = df_test.iloc[:,1:]
xDimTest = np.shape(X_test)[1]
X_test = X_test.values.reshape(-1,xDimTest)
y_test = df_test.Result.values

X_test = pca.transform(X_test)

Create Kfold splits

In [140]:
kfold = StratifiedKFold(n_splits=5, shuffle=True)

In [141]:
np.random.seed()

## MLP

Various properties

In [150]:
dropRate = 0.3
numBatch = 50
numEpoch = 120
learningRate = 3e-5

Base Single Layer Model

In [151]:
# MLP model
MLP = Sequential()
MLP.name = 'MLP'
MLP.add(Dense(200, input_dim=xDim, kernel_initializer='glorot_normal',activation = 'tanh'))
MLP.add(Dropout(dropRate))
# MLP.add(Dense(200,kernel_initializer='glorot_normal',activation = 'tanh'))
# MLP.add(Dropout(dropRate))
# MLP.add(Dense(200, kernel_initializer='glorot_normal',activation = 'tanh'))
# MLP.add(Dropout(dropRate))
MLP.add(Dense(1, kernel_initializer='glorot_normal', activation='sigmoid'))

# Compile model
adam = optimizers.Adam(lr=learningRate, amsgrad=True)
MLP.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [152]:
MLP.fit(X_train, y_train, epochs=numEpoch, batch_size=numBatch, verbose=1)

Epoch 1/120
504/504 [==============================] - 1s 1ms/step - loss: 0.7116 - acc: 0.6270
Epoch 2/120
504/504 [==============================] - 0s 52us/step - loss: 0.6862 - acc: 0.6250
Epoch 3/120
504/504 [==============================] - 0s 59us/step - loss: 0.6621 - acc: 0.6528
Epoch 4/120
504/504 [==============================] - 0s 59us/step - loss: 0.6560 - acc: 0.6528
Epoch 5/120
504/504 [==============================] - 0s 57us/step - loss: 0.6679 - acc: 0.6488
Epoch 6/120
504/504 [==============================] - 0s 51us/step - loss: 0.6545 - acc: 0.6488
Epoch 7/120
504/504 [==============================] - 0s 46us/step - loss: 0.6493 - acc: 0.6607
Epoch 8/120
504/504 [==============================] - 0s 59us/step - loss: 0.6280 - acc: 0.6766
Epoch 9/120
504/504 [==============================] - 0s 48us/step - loss: 0.6319 - acc: 0.6865
Epoch 10/120
504/504 [==============================] - 0s 63us/step - loss: 0.6063 - acc: 0.6726
Epoch 11/120
504/504 [========

504/504 [==============================] - 0s 65us/step - loss: 0.4735 - acc: 0.7659
Epoch 85/120
504/504 [==============================] - 0s 52us/step - loss: 0.4710 - acc: 0.7540
Epoch 86/120
504/504 [==============================] - 0s 58us/step - loss: 0.4611 - acc: 0.7679
Epoch 87/120
504/504 [==============================] - 0s 52us/step - loss: 0.4771 - acc: 0.7659
Epoch 88/120
504/504 [==============================] - 0s 53us/step - loss: 0.4631 - acc: 0.7679
Epoch 89/120
504/504 [==============================] - 0s 48us/step - loss: 0.4708 - acc: 0.7599
Epoch 90/120
504/504 [==============================] - 0s 48us/step - loss: 0.4716 - acc: 0.7639
Epoch 91/120
504/504 [==============================] - 0s 53us/step - loss: 0.4791 - acc: 0.7540
Epoch 92/120
504/504 [==============================] - 0s 46us/step - loss: 0.4620 - acc: 0.7599
Epoch 93/120
504/504 [==============================] - 0s 48us/step - loss: 0.4611 - acc: 0.7698
Epoch 94/120
504/504 [===========

In [153]:
MLP.evaluate(X_test, y_test)

378/378 [==============================] - 0s 826us/step


[0.47323935183267746, 0.7751322760783806]

## Random Forest

In [44]:
fore = RandomForestClassifier(criterion='entropy', n_estimators = 50, max_depth = 4, oob_score=True)

fore.fit(X_train, y_train)

y_pred =  fore.predict_proba(X_train)[:,1].reshape(-1,1)
LL = log_loss( y_train, y_pred)
print("Training set score: {:4}" .format(LL))
y_pred =  fore.predict_proba(X_test)[:,1].reshape(-1,1)
LL = log_loss( y_test, y_pred)
print("Validation set score: {:4}".format(LL))

Training set score: 0.3816087166876445
Validation set score: 0.4837797427646516


In [45]:
accTest = fore.score(X_test, y_test)
print("Validation accuracy: {:4}".format(accTest))

Validation accuracy: 0.7671957671957672


## Making predictions with model

### Extract data desired

In [154]:
data_dir = '../Final/'
df_sample_sub = pd.read_csv(data_dir + 'WSampleSubmissionStage2.csv')
data_file = data_dir + 'FinalMarchMadnessAdvStats.csv'
df_adv = pd.read_csv(data_file)
df_seeds = pd.read_csv(data_dir + 'WNCAATourneySeeds.csv')


n_test_games = len(df_sample_sub)

def get_year_t1_t2(ID):
    """Return a tuple with ints `year`, `team1` and `team2`."""
    return (int(x) for x in ID.split('_'))

def seed_to_int(seed):
    '''Get just the digits from the seeding. Return as int'''
    s_int = int(seed[1:3])
    return s_int

In [155]:
print('Loading data for submission test')

# Make the seeding an integer
df_seeds['seed_int'] = df_seeds.Seed.apply(seed_to_int)
df_seeds.drop(columns=['Seed'], inplace=True) # This is the string label
df_seeds.head()


T1_seed = []
T1_adv = []
T2_adv = []
T2_seed = []
for ii, row in df_sample_sub.iterrows():
    year, t1, t2 = get_year_t1_t2(row.ID)
    t1_seed = df_seeds[(df_seeds.TeamID == t1) & (df_seeds.Season == year)].seed_int.values[0]
    t2_seed = df_seeds[(df_seeds.TeamID == t2) & (df_seeds.Season == year)].seed_int.values[0]
    t1_adv = df_adv[(df_adv.TeamID == t1) & (df_adv.Season == year)].values[0]
    t2_adv = df_adv[(df_adv.TeamID == t2) & (df_adv.Season == year)].values[0]
    T1_seed.append(t1_seed)
    T1_adv.append(t1_adv)
    T2_seed.append(t2_seed)
    T2_adv.append(t2_adv)

T1_adv = [row[2:] for row in T1_adv]
T2_adv = [row[2:] for row in T2_adv]
T1_seed = np.reshape(T1_seed, [n_test_games,-1]).tolist()
T2_seed = np.reshape(T2_seed, [n_test_games, -1]).tolist()
X_pred = np.concatenate((T1_seed, T1_adv, T2_seed, T2_adv), axis=1)

df_subData = pd.DataFrame(np.array(X_pred).reshape(np.shape(X_pred)[0], np.shape(X_pred)[1]))

xDim = np.shape(df_subData)[1]
X_pred = df_subData.values.reshape(-1,xDim)

Loading data for submission test


In [48]:
np.shape(X_pred)

(2016, 28)

In [156]:
X_pred = pca.transform(X_pred)

In [157]:
preds = MLP.predict_proba(X_pred)

# df_sample_sub = pd.DataFrame()
# clipped_preds = np.clip(preds, 0.05, 0.95)
df_sample_sub.Pred = preds
df_sample_sub.shape

(2016, 2)

In [50]:
preds = fore.predict_proba(X_pred)

# df_sample_sub = pd.DataFrame()
# clipped_preds = np.clip(preds, 0.05, 0.95)
df_sample_sub.Pred = preds
df_sample_sub.shape

(2016, 2)

In [159]:
filename = 'MLP2'
save_dir = '../Final/'
c=0
ext = '.csv'
if os.path.exists(save_dir+filename+ext):
    while os.path.exists(filename+ext):
        c+=1
    filename = filename+'_'+str(c)
    df_sample_sub.to_csv(save_dir+filename+ext, index=False)
else:
    df_sample_sub.to_csv(save_dir+filename+ext, index=False)